# Día 4: Principios SOLID

## Descripción General

Los **principios SOLID** son cinco principios de diseño orientado a objetos que ayudan a crear software más mantenible, flexible y escalable. Fueron introducidos por Robert C. Martin y son fundamentales en el desarrollo de software profesional.

SOLID es un acrónimo de:
- **S**ingle Responsibility Principle
- **O**pen/Closed Principle
- **L**iskov Substitution Principle
- **I**nterface Segregation Principle
- **D**ependency Inversion Principle

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender los cinco principios SOLID
2. Aplicar cada principio SOLID en código Python
3. Identificar violaciones de los principios SOLID
4. Refactorizar código para cumplir con SOLID
5. Diseñar sistemas siguiendo los principios SOLID

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python Classes Tutorial](https://docs.python.org/3/tutorial/classes.html)**: Tutorial oficial sobre clases en Python

### Mejores Prácticas
- **[SOLID Principles in Python](https://realpython.com/solid-principles-python/)**: Guía completa sobre SOLID en Python

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente